# Parser for Orca Scans

In [26]:
import cclib
import pybel
import glob
import os
import pandas as pd

In [27]:
from chemreps.utils.molecule import Molecule
from math import sqrt
import re

numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts


def length(molecule, atomi, atomj):
    """
    Returns the length between two atoms
    Parameters
    -----------
    molecule : object
        molecule object
    atomi, atomj : int
        atoms
    Returns
    --------
    rij : float
        length between the two
    """
    x = molecule.xyz[atomi][0] - molecule.xyz[atomj][0]
    y = molecule.xyz[atomi][1] - molecule.xyz[atomj][1]
    z = molecule.xyz[atomi][2] - molecule.xyz[atomj][2]
    rij = sqrt((x ** 2) + (y ** 2) + (z ** 2))
    return rij

molfiles = ['HF', 'CO', 'H2', 'H2O', 'HCCH', 'H2CCH2', 'HCN', 
            'NH3', 'N2', 'CH4', 'CH3OH', 'C6H6-C', 'C6H6-H', 
            'biphenyl', 'ala-ala', 'aspartame', 'gly-gly']
largermol = ['biphenyl', 'ala-ala', 'aspartame', 'gly-gly']
stretches = ['FH', 'OC', 'HH', 'OH', 'CC', 'CC', 'NC', 'NH', 
             'NN', 'CH', 'OC', 'CC', 'CH', 'CC', 'NC', 'CC', 'NC']

In [28]:
elems = {1:'H', 6:'C', 7:'N', 8:'O', 9:'F'}

energies = []
for file in sorted(glob.iglob('*.out')):
    molecule = file.split('.')[0]
    data = cclib.io.ccread(file)
    for i in range(len(data.atomcoords)):
#         if i == 0:
#             pass
#         else:
        # get geometery and energy 
        geom = data.atomcoords[i]
        energy = data.scfenergies[i]
        natom = data.natom
        # write xyz file
        with open('stretch/{}/xyz/{}.xyz'.format(molecule, i), 'w') as fxyz:
            print(natom, file=fxyz)
            print('Energy: \t{}'.format(data.scfenergies[i]), file=fxyz)
            for j in range(natom):
                sym = elems[data.atomnos[j]]
                x = geom[j][0]
                y = geom[j][1]
                z = geom[j][2]
                print('{} \t\t{} \t\t{} \t\t{}'.format(sym, x, y, z), file=fxyz)

        # open with pybel in order to make sdf
        mol = next(pybel.readfile('xyz', 'stretch/{}/xyz/{}.xyz'.format(molecule, i)))
        # rewrite xyz to make cleaner
        output = pybel.Outputfile('xyz', 'stretch/{}/xyz/{}.xyz'.format(molecule, i), overwrite=True)
        output.write(mol)
        output.close()
        # write sdf file
        output = pybel.Outputfile('sdf', 'stretch/{}/sdf/{}.sdf'.format(molecule, i), overwrite=True)
        output.write(mol)
        output.close()

        # grab bond length and stretch
        btype = stretches[molfiles.index(molecule)]
        current_molecule = Molecule('stretch/{}/sdf/{}.sdf'.format(molecule, i))
        doubcount = 0
        if molecule in largermol:
            if molecule == 'biphenyl':
                ai = 2
                aj = 11
            if molecule == 'ala-ala':
                ai = 2
                aj = 11
            if molecule == 'gly-gly':
                ai = 2
                aj = 8
            if molecule == 'aspartame':
                ai = 11
                aj = 10
            atomi = current_molecule.sym[ai]
            atomj = current_molecule.sym[aj]
            zi = current_molecule.at_num[ai]
            zj = current_molecule.at_num[aj]

            if zj > zi:
                # swap ordering
                atomi, atomj = atomj, atomi
                bond = "{}{}".format(atomi, atomj)

            rij = length(current_molecule, ai, aj)

        else:
            for k in range(current_molecule.n_atom):
                for l in range(k, current_molecule.n_atom):
                    atomi = current_molecule.sym[k]
                    atomj = current_molecule.sym[l]
                    zi = current_molecule.at_num[k]
                    zj = current_molecule.at_num[l]
                    if k == l:
                        mii = 0.5 * zi ** 2.4
                    else:
                        if zj > zi:
                            # swap ordering
                            atomi, atomj = atomj, atomi
                        bond = "{}{}".format(atomi, atomj)
                        if bond == btype:
                            doubcount += 1
                            if doubcount == 6:
                                # rij = sqrt((xi - xj)^2 + (yi - yj)^2 + (zi - zj)^2)
                                rij = length(current_molecule, k, l)

        # store in dict to append to list
        d = {}
        d.update({'name': molecule})
        d.update({'point': i})
        d.update({'bond': btype})
        d.update({'length': rij})
        d.update({'energy': energy})
        energies.append(d)
            

In [29]:
df = pd.DataFrame(energies)
df

,name,point,bond,length,energy
0,C6H6-C,0,CC,0.884612,-6202.761889
1,C6H6-C,1,CC,0.984610,-6259.129633
2,C6H6-C,2,CC,1.084608,-6290.919938
3,C6H6-C,3,CC,1.184605,-6307.793560
4,C6H6-C,4,CC,1.284603,-6315.569917
...,...,...,...,...,...
777,gly-gly,41,NC,4.937205,-13387.905928
778,gly-gly,42,NC,5.037199,-13387.878640
779,gly-gly,43,NC,5.137189,-13387.852719
780,gly-gly,44,NC,5.237214,-13387.828043


In [30]:
df.to_csv('../data/dft-data.csv', index=False)

#### Sucrose was organized a bit differently requiring a change in parsing

In [31]:
molfiles = ['HF', 'CO', 'H2', 'H2O', 'HCCH', 'H2CCH2', 'HCN', 
            'NH3', 'N2', 'CH4', 'CH3OH', 'C6H6-C', 'C6H6-H', 
            'biphenyl', 'ala-ala', 'aspartame', 'gly-gly', 'sucrose']
largermol = ['biphenyl', 'ala-ala', 'aspartame', 'gly-gly', 'sucrose']
stretches = ['FH', 'OC', 'HH', 'OH', 'CC', 'CC', 'NC', 'NH', 
             'NN', 'CH', 'OC', 'CC', 'CH', 'CC', 'NC', 'CC', 'NC', 'CC']

In [32]:
elems = {1:'H', 6:'C', 7:'N', 8:'O', 9:'F'}

energies = []
for file in sorted(glob.iglob('scan/sucrose-bond/*.out.bz2'), key=numericalSort):
    molecule = 'sucrose'
    pt = int(file.split('.')[0].split('bond/sucrose-')[1])
    data = cclib.io.ccread(file)

    # get geometery and energy 
    geom = data.atomcoords[0]
    energy = data.scfenergies[0]
    natom = data.natom
    # write xyz file
    with open('stretch/{}/xyz/{}.xyz'.format(molecule, pt), 'w') as fxyz:
        print(natom, file=fxyz)
        print('Energy: \t{}'.format(data.scfenergies[0]), file=fxyz)
        for j in range(natom):
            sym = elems[data.atomnos[j]]
            x = geom[j][0]
            y = geom[j][1]
            z = geom[j][2]
            print('{} \t\t{} \t\t{} \t\t{}'.format(sym, x, y, z), file=fxyz)

    # open with pybel in order to make sdf
    mol = next(pybel.readfile('xyz', 'stretch/{}/xyz/{}.xyz'.format(molecule, pt)))
    # rewrite xyz to make cleaner
    output = pybel.Outputfile('xyz', 'stretch/{}/xyz/{}.xyz'.format(molecule, pt), overwrite=True)
    output.write(mol)
    output.close()
    # write sdf file
    output = pybel.Outputfile('sdf', 'stretch/{}/sdf/{}.sdf'.format(molecule, pt), overwrite=True)
    output.write(mol)
    output.close()
            
    # grab bond length and stretch
    btype = stretches[molfiles.index(molecule)]
    current_molecule = Molecule('stretch/{}/sdf/{}.sdf'.format(molecule, pt))
    doubcount = 0

    if molecule in largermol:
        if molecule == 'biphenyl':
            ai = 2
            aj = 11
        if molecule == 'ala-ala':
            ai = 2
            aj = 11
        if molecule == 'gly-gly':
            ai = 2
            aj = 8
        if molecule == 'aspartame':
            ai = 11
            aj = 10
        if molecule == 'sucrose':
            ai = 0
            aj = 22
            
        atomi = current_molecule.sym[ai]
        atomj = current_molecule.sym[aj]
        zi = current_molecule.at_num[ai]
        zj = current_molecule.at_num[aj]

        if zj > zi:
            # swap ordering
            atomi, atomj = atomj, atomi
            bond = "{}{}".format(atomi, atomj)

        rij = length(current_molecule, ai, aj)

    else:
        for k in range(current_molecule.n_atom):
            for l in range(k, current_molecule.n_atom):
                atomi = current_molecule.sym[k]
                atomj = current_molecule.sym[l]
                zi = current_molecule.at_num[k]
                zj = current_molecule.at_num[l]
                if k == l:
                    mii = 0.5 * zi ** 2.4
                else:
                    if zj > zi:
                        # swap ordering
                        atomi, atomj = atomj, atomi
                    bond = "{}{}".format(atomi, atomj)
                    if bond == btype:
                        doubcount += 1
                        if doubcount == 6:
                            # rij = sqrt((xi - xj)^2 + (yi - yj)^2 + (zi - zj)^2)
                            rij = length(current_molecule, k, l)

    # store in dict to append to list
    d = {}
    d.update({'name': molecule})
    d.update({'point': pt})
    d.update({'bond': btype})
    d.update({'length': rij})
    d.update({'energy': energy})
    energies.append(d)
            

In [33]:
df = pd.DataFrame(energies)
df

,name,point,bond,length,energy
0,sucrose,0,CC,0.929524,-35296.599113
1,sucrose,1,CC,1.029507,-35303.357953
2,sucrose,2,CC,1.129509,-35307.015852
3,sucrose,3,CC,1.229547,-35308.864051
4,sucrose,4,CC,1.329549,-35309.654299
5,sucrose,5,CC,1.429532,-35309.837182
6,sucrose,6,CC,1.529499,-35309.678647
7,sucrose,7,CC,1.629573,-35309.336600
8,sucrose,8,CC,1.729539,-35308.905036
9,sucrose,9,CC,1.829522,-35308.440359


In [34]:
df.to_csv('../data/sucrose-data.csv', index=False)